# Simulate Annealing for Single Instances

In [ ]:
using QAOA, Distributions, Interpolations, LinearAlgebra
using HDF5, Printf
using PythonPlot
# PythonPlot.matplotlib.style.use("./paper.mplstyle")
PythonPlot.rc("axes", prop_cycle=PythonPlot.matplotlib.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

PATH = "../";
# PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.8f", f)

In [ ]:
N = 9

patterns_dict = Dict(
    9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
    11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
    13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
    15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
    17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
    19 => r"random_SK_instance_N_19_seed_(\d+)\.h5"
);

subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
# N = 9
seed = 23583;
seed = 22824;

In [ ]:
J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
mf_problem = Problem(0, J_mat);

λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

In [ ]:
npts = 2048
# npts = 8192
coarse_times = range(0, 1, npts + 1);

## Exact gap

In [ ]:
# T_final = 32768
T_final = 32000
tol = 1e-8;

In [ ]:
exact_times = range(0, 1, 33);

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap) 
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap) 
gaploc = exact_times[gap_idx] 
gaploc |> println

In [ ]:
small_idxs = findall(x -> x < 0.05, gap ./ (2pi)) 
gap_interval = exact_times[small_idxs];

In [ ]:
bogo_spec = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32));

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(bogo_spec)[1]
    plot(exact_times[2:end], bogo_spec[i, :], "--C1", lw=1.5, alpha=1, ms=2)
    plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw=1, alpha=1, ms=2)    
end
plot([], [], "-k", lw=0.75, label="Exact")
plot([], [], "--C1", lw=1.5, label="Bogoliubov")
xlim(0., 1.)
ylim(-0., 2)
xlabel("\$s\$")
ylabel("\$E_\\alpha - E_0\$")
legend(frameon=false)
tight_layout()
display(gcf())
PythonPlot.close();

### Eigenstates

In [ ]:
all_eigvecs = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_lowest_eigvecs");

In [ ]:
H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings);

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]

overlap_01_x = [first_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_01_z = [first_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_02_x = [second_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_02_z = [second_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)];

In [ ]:
figure(figsize=(4, 3))
ax = subplot(111)
plot(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label="\$\\alpha = 1\$")
plot(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])), label="\$\\alpha = 2\$", lw=2)
xlim(0., 1.)
ylim(0, 5)
xlabel("\$s\$")
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")

legend(frameon=false)

tight_layout()
display(gcf())
PythonPlot.close();

## Annealing

In [ ]:
# fail
T_anneal = 32.
p = 512

T_anneal = 256.
p = 2^13

T_anneal = 2.0^15
p = 2^17

linear_schedule(t) = t / T_anneal

# second-order schedule
τ = T_anneal / p
γ = τ .* ((1:p) .- 1/2) ./ p |> collect
β = τ .* (1 .- (1:p) ./ p) |> collect
β[p] = τ / (4 * p);

In [ ]:
# annealing_problem = Problem(p, zeros(N), J_mat)
annealing_problem = Problem(p, J_mat)

In [ ]:
H_Z = -hamiltonian(0, 1, annealing_problem.local_fields, annealing_problem.couplings);

In [ ]:
# probs = anneal(annealing_problem, linear_schedule, T_anneal);

In [ ]:
using Yao
beta_and_gamma = vcat(β, γ)
circ = QAOA.circuit(annealing_problem)
circ = QAOA.dispatch_parameters!(circ, annealing_problem, beta_and_gamma)
probs = Yao.uniform_state(Yao.nqubits(circ)) |> circ |> Yao.probs;

In [ ]:
figure(figsize=(3, 2))
plot(probs, label=@sprintf("\$T = %0.f\$", T_anneal))
xlim(0, 2^annealing_problem.num_qubits)
ylim(0, 1)
legend(frameon=false)
tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

In [ ]:
max_prob = maximum(probs)
max_prob |> println
sol_idxs = findall(x -> x == max_prob, probs)

In [ ]:
max_prob_2 = maximum(filter(x -> x != max_prob, probs))
max_prob_2 |> println
sol_idxs_2 = findall(x -> x == max_prob_2, probs)

In [ ]:
bitstrings = digits.(0:2^annealing_problem.num_qubits-1, base=2, pad=annealing_problem.num_qubits)
vec_dict = Dict(1 => [1, 0], -1 => [0, 1]);

In [ ]:
1 .- 2bitstrings[sol_idxs[1]] |> println
1 .- 2bitstrings[sol_idxs_2[1]] |> println

In [ ]:
sol_vec_1 = reduce(kron, [vec_dict[1 - 2z] for z in bitstrings[sol_idxs[1]]]);
sol_vec_2 = reduce(kron, [vec_dict[1 - 2z] for z in bitstrings[sol_idxs_2[1]]]);

In [ ]:
sol_vec_1' * H_Z * sol_vec_1 |> println
sol_vec_2' * H_Z * sol_vec_2 |> println

In [ ]:
max_prob + max_prob_2